### Fetching and loading the data

In [29]:
import kaggle
import pandas as pd
from pathlib import Path

def fetch_load_kaggle_dataset(dataset, fetch=False):
    if fetch:
        kaggle.api.authenticate()
        kaggle.api.dataset_download_files(dataset, path='../datasets', unzip=True)
    return pd.read_csv(Path("../datasets/intermittent-renewables-production-france.csv"))

electricity_production_data = fetch_load_kaggle_dataset('henriupton/wind-solar-electricity-production')

Scikit-learn does not support removing rows in a pipeline, so we should do this prior to constructing our pipeline. We will filter samples with NaN values or "Solar" Source.

In [30]:
electricity_production_data = electricity_production_data.dropna()
electricity_production_data[electricity_production_data.isnull().any(axis=1)]

,Date and Hour,Date,StartHour,EndHour,Source,Production,dayOfYear,dayName,monthName


In [31]:
drop_indexes = electricity_production_data[electricity_production_data["Source"] == "Solar"].index
electricity_production_data.drop(drop_indexes, inplace=True)

### Split the dataset, and extract the labels

In [32]:
from sklearn.model_selection import train_test_split

strat_train_set, strat_test_set = train_test_split(electricity_production_data, test_size=0.2, stratify=electricity_production_data["monthName"])

electricity_production_data = strat_train_set.drop("Production", axis=1)
electricity_production_data_labels = strat_train_set["Production"].copy()

### Feature transformations and transformation pipelines

Involving creating pipelines to preprocess our attributes, and use feature engineering to construct new cluster similarity features.
- Encoding cyclical attributes
- Encoding ordinal attributes
- Dropping redundant attributes
- Normalizing non-cyclical attributes (min-max & standard scalers)
- Creating new cluster similarity measures (feature engineering)

In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np

def encode_cyc(X):
    vals = {}
    max_val = X.iloc[:, 0].max()
    for dist_val in X.iloc[:, 0].unique():
        vals[dist_val] = (np.sin(2 * np.pi * dist_val / max_val),
                          np.cos(2 * np.pi * dist_val / max_val))

    return pd.DataFrame([*X.iloc[:, 0].apply(lambda v: vals[v])])

def encode_get_seconds(X):
    return pd.to_timedelta(X.iloc[:, 0]).apply(lambda td: td.seconds).to_frame() 

def encode_get_year(X):
    return pd.to_datetime(X.iloc[:, 0]).apply(lambda dt: dt.year).to_frame()

def encode_ordinal(X):
    return pd.to_datetime(X.iloc[:, 0]).apply(lambda dt: dt.toordinal()).to_frame()

def encode_positional(X):
    min_attr = X.iloc[:, 0].min()
    return X.iloc[:, 0].apply(lambda j: j - min_attr + 1).to_frame()

def encode_numeric_label(X, labels):
    return X.iloc[:, 0].apply(lambda n: labels.index(n) + 1).to_frame()

class ClusterSimilarityMeasure(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=5, gamma=1.0, random_state=42, sample_weight_label_scale=.03):
        self.n_clusters = n_clusters
        self.sample_weight_label_scale = sample_weight_label_scale
        self.gamma = gamma
        self.random_state = random_state
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)

    def fit(self, X, y=None):
        exp_scaled_labels = np.exp(y * self.sample_weight_label_scale)
        self.kmeans_.fit(X, sample_weight=exp_scaled_labels)
        return self

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]
    
class ScalerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, scaler):
        self.scaler = scaler
        
    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self
    
    def transform(self, X, y=None):
        return self.scaler.transform(X)
    
    def get_feature_names_out(self, input_features=None):
        if input_features:
            return input_features
        return self.feature_names_in_
            


In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config

set_config(display="diagram")

# column name funcs
def sin_cos_column_names(t, fn):
    return \
        [suffix_column_name(fn[0], "_sin"),
         suffix_column_name(fn[0], "_cos")]

def suffix_column_name(cn, suffix):
    return cn + suffix

# Generic cyclical and positional transformers
cyc_transformer = FunctionTransformer(encode_cyc, feature_names_out=sin_cos_column_names)
pos_transformer = FunctionTransformer(
    encode_positional,
    feature_names_out=lambda t, fn: [suffix_column_name(fn[0], "_pos")]
)

# Cluster similarity measure - sample weights set inside estimator fit method
cluster_simil_pipeline = make_pipeline(
    cyc_transformer,
    ClusterSimilarityMeasure(n_clusters=3, sample_weight_label_scale=0.03),
)

# dayOfYear
day_of_year_pipeline = make_pipeline(
    cyc_transformer
)

# StartHour
start_hour_pipeline = make_pipeline(
    FunctionTransformer(
        encode_get_seconds, 
        feature_names_out=lambda t, fn: [suffix_column_name(fn[0], "Seconds")]
    ),
    cyc_transformer
)

# monthName
months = ["January", "February", "March",
          "April", "May", "June",
          "July", "August", "September",
          "October", "November", "December"]
month_name_pipeline = make_pipeline(
    FunctionTransformer(
        encode_numeric_label, 
        kw_args=dict(labels=months), 
        feature_names_out=lambda t, fn: [suffix_column_name(fn[0], "Numeric")]
    ),
    cyc_transformer
)

# Year and Date
year_pipeline = make_pipeline(
    FunctionTransformer(
        encode_get_year,
        feature_names_out=lambda t, fn: [suffix_column_name(fn[0], "Year")]
    ),
    pos_transformer,
    ScalerWrapper(StandardScaler())
)

date_pipeline = make_pipeline(
    FunctionTransformer(
        encode_ordinal,
        feature_names_out=lambda t, fn: [suffix_column_name(fn[0], "Ordinal")]
    ),
    pos_transformer,
    ScalerWrapper(MinMaxScaler(feature_range=(-1, 1)))
)

preprocessing = ColumnTransformer([
    ("day_of_year_pipeline", day_of_year_pipeline, ["dayOfYear"]),
    ("day_of_year_cluster_simil_pipeline", cluster_simil_pipeline, ["dayOfYear"]),
    ("start_hour_pipeline", start_hour_pipeline, ["StartHour"]),
    ("month_name_pipeline", month_name_pipeline, ["monthName"]),
    ("year_pipeline", year_pipeline, ["Date"]),
    ("date_pipeline", date_pipeline, ["Date"]),
], remainder="drop")

# labels needed for calculating cluster sample weights
electricity_production_data_prepared = (
    preprocessing.fit_transform(electricity_production_data, electricity_production_data_labels))
electricity_production_data_prepared

array([[ 0.49254807, -0.87028524,  0.35704969, ..., -1.        ,
        -0.3097689 , -0.18808777],
       [-0.56534467,  0.82485477,  0.04431255, ...,  0.8660254 ,
         0.67133684,  0.6630094 ],
       [-0.7221166 ,  0.69177136,  0.03231239, ...,  0.8660254 ,
        -0.3097689 ,  0.07210031],
       ...,
       [ 0.35275209, -0.93571682,  0.2696622 , ..., -1.        ,
        -1.29087465, -0.7476489 ],
       [ 0.77876393,  0.6273171 ,  0.64936495, ...,  0.5       ,
         1.65244259,  0.79780564],
       [-0.33663728,  0.9416344 ,  0.07015243, ...,  1.        ,
         0.67133684,  0.68652038]])

In [35]:
preprocessing.get_feature_names_out()

array(['day_of_year_pipeline__dayOfYear_sin',
       'day_of_year_pipeline__dayOfYear_cos',
       'day_of_year_cluster_simil_pipeline__Cluster 0 similarity',
       'day_of_year_cluster_simil_pipeline__Cluster 1 similarity',
       'day_of_year_cluster_simil_pipeline__Cluster 2 similarity',
       'start_hour_pipeline__StartHourSeconds_sin',
       'start_hour_pipeline__StartHourSeconds_cos',
       'month_name_pipeline__monthNameNumeric_sin',
       'month_name_pipeline__monthNameNumeric_cos',
       'year_pipeline__DateYear_pos', 'date_pipeline__DateOrdinal_pos'],
      dtype=object)

### Selecting, training, fine-tuning and testing models

Here we can trial various models and fine tune our hyperparameters, run inferences against our model and measure performance.

Candidate algorithms to test
- LinearRegressor
- SGDRegressor
- DecisionTreeRegressor
- RandomForestRegressor

In [36]:
from sklearn.linear_model import LinearRegression, SGDRegressor

lin_reg = make_pipeline(preprocessing, LinearRegression())
sgd_reg = make_pipeline(preprocessing, SGDRegressor())
lin_reg.fit(electricity_production_data, electricity_production_data_labels)
sgd_reg.fit(electricity_production_data, electricity_production_data_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('day_of_year_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function sin_cos_column_names at 0x12a8ca4c0>,
                                                                                       func=<function encode_cyc at 0x11fb884c0>))]),
                                                  ['dayOfYear']),
                                                 ('day_of_year_cluster_simil_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   F...
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x14b6e1ca0>,
                                                                                       func=<function encode_ordinal at 0x14b490e50>)),
                                                                  ('functiontransformer-2',
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x12aedad30>,
                                                                                       func=<function encode_positional at 0x14b6e1040>)),
                                                                  ('scalerwrapper',
                                                                   ScalerWrapper(scaler=MinMaxScaler(feature_range=(-1,
                                                                                                                    1))))]),
                                                  ['Date'])])),
                ('sgdregressor', SGDRegressor())])

In [37]:
electricity_production_data_predictions = sgd_reg.predict(electricity_production_data)
electricity_production_data_predictions[:10]

array([3430.50393614, 6114.52979121, 5330.98789154, 3518.54982181,
       4831.30445289, 6763.12529028, 4380.10531945, 5888.92482498,
       3334.7748963 , 3525.26007167])

In [38]:
# error ratios between predicted values and labrls
error_ratios = electricity_production_data_predictions[:10] / electricity_production_data_labels.iloc[:10].values - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

91.3%, -2.2%, 72.5%, 989.3%, -37.1%, 14.3%, 153.2%, 115.1%, -60.4%, 36.1%


In [39]:
from sklearn.metrics import mean_squared_error

# calculate root mean squared error of the model on entire training set
lin_mse = mean_squared_error(electricity_production_data_labels, electricity_production_data_predictions, squared=False)
lin_mse

3162.6093990860277

The above shows a typical prediction error of ~3136 MWh. Let's test with other regressors to observe difference in the mean squared error.

In [40]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(electricity_production_data, electricity_production_data_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('day_of_year_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function sin_cos_column_names at 0x12a8ca4c0>,
                                                                                       func=<function encode_cyc at 0x11fb884c0>))]),
                                                  ['dayOfYear']),
                                                 ('day_of_year_cluster_simil_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   F...
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x14b6e1ca0>,
                                                                                       func=<function encode_ordinal at 0x14b490e50>)),
                                                                  ('functiontransformer-2',
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x12aedad30>,
                                                                                       func=<function encode_positional at 0x14b6e1040>)),
                                                                  ('scalerwrapper',
                                                                   ScalerWrapper(scaler=MinMaxScaler(feature_range=(-1,
                                                                                                                    1))))]),
                                                  ['Date'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [41]:
electricity_production_data_predictions = tree_reg.predict(electricity_production_data)

tree_rmse = mean_squared_error(electricity_production_data_labels, electricity_production_data_predictions, squared=False)
tree_rmse

391.23854235448255

As we can see, we have a much lower RMSE for this algorithm. We can't know whether the algorithm is better suited because it can find non-linear relationships in our data, or if the model is over-fitting the data. We'll have to observe our model performance on unseen data using validation and test sets.

Sklearn has a k-fold cross validation feature we can use to train and evaluate the decision tree model using 10 non-overlapping subsets of the training set called folds. The 10 iterations using 9 folds for training and a single fold for evaluation will be output in an array. 

Here we're using a utility function as a scoring function, rather than a cost function, so we should negate the output to find the RMSEs.

In [42]:
from sklearn.model_selection import cross_val_score

folds = 10
tree_rmses = -cross_val_score(tree_reg, electricity_production_data, electricity_production_data_labels, scoring="neg_root_mean_squared_error", cv=folds, error_score="raise")
pd.Series(tree_rmses).describe()

count      10.000000
mean     1455.685871
std       468.106879
min      1156.704261
25%      1224.532733
50%      1260.236954
75%      1317.232057
max      2566.056433
dtype: float64

Here, our training error is low, but our validation error is a bit higher, suggesting that the model might be over-fitting. Using multiple validation sets here allows us to estimate the model performance as well as the measure of how precise the estimate is (standard deviation). This involves training the model multiple times, which sometimes isn't always feasible.

In [43]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

forest_reg = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42, max_features=4)),
])
forest_rmses = -cross_val_score(forest_reg, electricity_production_data, electricity_production_data_labels, scoring="neg_root_mean_squared_error", cv=folds)
pd.Series(forest_rmses).describe()

count      10.000000
mean     1116.034934
std       260.489400
min       945.254658
25%       978.754766
50%      1006.620683
75%      1067.013652
max      1764.291279
dtype: float64

Now we've tested a few different models, we can use the cross validation with the `GridSearchCV` class to tune our hyperparameters and find an optimal combination of parameters for the model. Note for the sample weight label scale, we will try testing a set of continuous values within a set range, with equal likeliness all values in this range might be optimal.

In [44]:
from sklearn.model_selection import GridSearchCV

# tune preprocessing and model hyperparameters at the same time
param_grid = [
    {"preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters": [4, 5, 6],
     "preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale": [.02, .023, .025],
     "random_forest__max_features": [3, 4, 5]},
]

grid_search = GridSearchCV(forest_reg, param_grid, scoring="neg_root_mean_squared_error", cv=3)
grid_search.fit(electricity_production_data, electricity_production_data_labels)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('day_of_year_pipeline',
                                                                         Pipeline(steps=[('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<function sin_cos_column_names at 0x12a8ca4c0>,
                                                                                                              func=<function encode_cyc at 0x11fb884c0>))]),
                                                                         ['dayOfYear']),
                                                                        ('day_of_year_cluster_simil_pipeline',
                                                                         Pipeline(steps=[...
                                        RandomForestRegressor(max_features=4,
                                                              random_state=42))]),
             param_grid=[{'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters': [4,
                                                                                                                      5,
                                                                                                                      6],
                          'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale': [0.02,
                                                                                                                                     0.023,
                                                                                                                                     0.025],
                          'random_forest__max_features': [3, 4, 5]}],
             scoring='neg_root_mean_squared_error')

In [45]:
grid_search.best_params_

{'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters': 6,
 'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale': 0.025,
 'random_forest__max_features': 4}

In [46]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('day_of_year_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function sin_cos_column_names at 0x12a8ca4c0>,
                                                                                       func=<function encode_cyc at 0x11fb884c0>))]),
                                                  ['dayOfYear']),
                                                 ('day_of_year_cluster_simil_pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   Funct...
                                                                                       func=<function encode_ordinal at 0x14b490e50>)),
                                                                  ('functiontransformer-2',
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x12aedad30>,
                                                                                       func=<function encode_positional at 0x14b6e1040>)),
                                                                  ('scalerwrapper',
                                                                   ScalerWrapper(scaler=MinMaxScaler(feature_range=(-1,
                                                                                                                    1))))]),
                                                  ['Date'])])),
                ('random_forest',
                 RandomForestRegressor(max_features=4, random_state=42))])

And it's possible for us to check the hyperparameter combination evaluation scores tested during the above grid search by accessing `grid_search.cv_results_`. This dictionary can be converted to a DataFrame.

In [47]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

# change column names and show rmse as -score
cv_res = cv_res[[
    "param_preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters",
    "param_preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale",
    "param_random_forest__max_features",
    "split0_test_score", "split1_test_score", "split2_test_score", "mean_test_score"]]
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = ["n_clusters", "sample_weight_label_scale", "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)

cv_res.head()

,n_clusters,sample_weight_label_scale,max_features,split0,split1,split2,mean_test_rmse
25,6,0.025,4,1052,1059,1084,1065
22,6,0.023,4,1054,1061,1091,1069
26,6,0.025,5,1059,1058,1095,1071
13,5,0.023,4,1056,1061,1097,1071
7,4,0.025,4,1053,1063,1098,1071


Now let's suppose we want to do the same using a `RandomizedSearchCV`. From this, we can search a larger number of combinations for continuous or discrete valued parameters, identify hyperparameters that might not contribute much, and run for a specific number of iterations for flexibility even in a larger search space.

In [48]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# further improvements on the following could involve exploring HalvingRandomSearchCV
param_distributions = {
    "preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters": randint(2, 10),
    "preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale": uniform(.018, .022),
    "random_forest__max_features": randint(3, 6)
}

rnd_search = RandomizedSearchCV(
    forest_reg, param_distributions=param_distributions, n_iter=15, cv=3,
    scoring="neg_root_mean_squared_error", random_state=42
)
rnd_search.fit(electricity_production_data, electricity_production_data_labels)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('day_of_year_pipeline',
                                                                               Pipeline(steps=[('functiontransformer',
                                                                                                FunctionTransformer(feature_names_out=<function sin_cos_column_names at 0x12a8ca4c0>,
                                                                                                                    func=<function encode_cyc at 0x11fb884c0>))]),
                                                                               ['dayOfYear']),
                                                                              ('day_of_year_cluster_simil_pipeline',
                                                                               Pipeline(s...
                   param_distributions={'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters': <scipy.stats._distn_infrastructure.rv_frozen object at 0x14b6e24c0>,
                                        'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale': <scipy.stats._distn_infrastructure.rv_frozen object at 0x12b392940>,
                                        'random_forest__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x14b6e2730>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [49]:
rnd_search.best_params_

{'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__n_clusters': 6,
 'preprocessing__day_of_year_cluster_simil_pipeline__clustersimilaritymeasure__sample_weight_label_scale': 0.027910983543329944,
 'random_forest__max_features': 4}

From our RandomSearchCV, we can see assess the relative importance of each feature when making accurate predictions.

In [50]:
model = rnd_search.best_estimator_
relative_feature_importance = model["random_forest"].feature_importances_ 
sorted(zip(relative_feature_importance, model["preprocessing"].get_feature_names_out()), reverse=True)

[(0.2038900152457571, 'date_pipeline__DateOrdinal_pos'),
 (0.09689740438952527,
  'day_of_year_cluster_simil_pipeline__Cluster 4 similarity'),
 (0.0918403980712294, 'day_of_year_pipeline__dayOfYear_cos'),
 (0.08027996042399588, 'year_pipeline__DateYear_pos'),
 (0.0780239086432657, 'start_hour_pipeline__StartHourSeconds_sin'),
 (0.06745944310600577,
  'day_of_year_cluster_simil_pipeline__Cluster 2 similarity'),
 (0.06646745388671078,
  'day_of_year_cluster_simil_pipeline__Cluster 0 similarity'),
 (0.05866836488616485,
  'day_of_year_cluster_simil_pipeline__Cluster 3 similarity'),
 (0.05828673989816628,
  'day_of_year_cluster_simil_pipeline__Cluster 5 similarity'),
 (0.0556449234955637,
  'day_of_year_cluster_simil_pipeline__Cluster 1 similarity'),
 (0.0534446002469151, 'day_of_year_pipeline__dayOfYear_sin'),
 (0.05331526601315618, 'start_hour_pipeline__StartHourSeconds_cos'),
 (0.02542249923489581, 'month_name_pipeline__monthNameNumeric_cos'),
 (0.010359022458648125, 'month_name_pipelin

If we considered dropping any features here, we might find that the monthName has very little to contribute, and so could be dropped. The `SelectFromModel` transformer's transform method can be used to automatically drop less useful features after model training. Let's now look at evaluating our model against unseen data in the test set.

In [51]:
electricity_production_data_test = strat_test_set.drop("Production", axis=1)
electricity_production_data_test_labels = strat_test_set["Production"].copy()

electricity_production_data_test_predictions = model.predict(electricity_production_data_test)
test_rmse = mean_squared_error(
    electricity_production_data_test_labels, 
    electricity_production_data_test_predictions, 
    squared=False)
test_rmse

1027.6846607912303

With our test error so close to our training error, it suggests we have a model that is well suited to generalizing to new unseen data. Now let's calculate a 95% confidence interval for the generalization error and see the interval size.

In [52]:
from scipy import stats

confidence = .95
squared_errors = (electricity_production_data_test_predictions - electricity_production_data_test_labels) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([ 971.42974765, 1081.01608195])

### Save the final model

We can then go on to export the model and deploy using some cloud deployment solution such as GCS or AWS SageMaker, etc.

In [ ]:
import joblib

# export the model
joblib.dump(model, "wind_power_production_model.pk1")

# reload the model
reload_model = joblib.load("wind_power_production_model.pk1")